In [1]:
import gym
import numpy as np
import pandas as pd
import json
# 打印gym版本
print(gym.__version__)

0.25.0


In [2]:
env = gym.make("CartPole-v1")
print("Created env:", env)

Created env: <TimeLimit<OrderEnforcing<StepAPICompatibility<PassiveEnvChecker<CartPoleEnv<CartPole-v1>>>>>>


/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
state = env.reset()
print("The starting state is:", state)

The starting state is: [ 0.04202396 -0.03643372 -0.00041004 -0.02315033]


In [4]:
state = env.reset()
print("The starting state is:", state)

The starting state is: [-0.04874318  0.03112491 -0.04214102 -0.00834236]


In [5]:
def random_rollout(env):
    state = env.reset()
    
    done = False
    cumulative_reward = 0

    # Keep looping as long as the simulation has not finished.
    while not done:
        # Choose a random action (either 0 or 1).
        action = np.random.choice([0, 1])
        
        # Take the action in the environment.
        state, reward, done, _ = env.step(action)
        
        # Update the cumulative reward.
        cumulative_reward += reward
    
    # Return the cumulative reward.
    return cumulative_reward  

In [6]:
reward = random_rollout(env)
print(reward)

46.0


/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [7]:
def rollout_policy(env, policy):
    state = env.reset()
    
    done = False
    cumulative_reward = 0

    while not done:
        action = policy(state)  # Choose the action using the policy.
        
        state, reward, done, _ = env.step(action)
        
        cumulative_reward += reward
    
    return cumulative_reward

def sample_policy1(state):
    return 0 if state[0] < 0 else 1

def sample_policy2(state):
    return 1 if state[0] < 0 else 0

reward1 = np.mean([rollout_policy(env, sample_policy1) for _ in range(100)])
reward2 = np.mean([rollout_policy(env, sample_policy2) for _ in range(100)])

print('The first sample policy got an average reward of {}.'.format(reward1))
print('The second sample policy got an average reward of {}.'.format(reward2))

assert 5 < reward1 < 15, ('Make sure that rollout_policy computes the action '
                          'by applying the policy to the state.')
assert 25 < reward2 < 35, ('Make sure that rollout_policy computes the action '
                           'by applying the policy to the state.')


The first sample policy got an average reward of 9.41.
The second sample policy got an average reward of 30.43.


In [8]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.logger import pretty_print
import ray
print(ray.__version__)

algo = (
    PPOConfig()
    .env_runners(num_env_runners=1) # env_runners is the number of actors that the PPO algorithm will create to run environments in parallel.
    .resources(num_gpus=0) # num_gpus is the number of GPUs that the PPO algorithm will use.
    .environment(env="CartPole-v1") # environment is the name of the environment that the PPO algorithm will run in.
    .build() # build() is a method that creates the PPO algorithm object.
)

for i in range(10):
    result = algo.train()
    print(pretty_print(result))

    if i % 5 == 0:
        checkpoint_dir = algo.save().checkpoint.path
        print(f"Checkpoint saved in directory {checkpoint_dir}")

/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:521: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/tune/logg

2.21.0


2024-05-13 23:03:55,507	INFO worker.py:1740 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8267 
/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/rllib/models/catalog.py:895: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  prep = cls(observation_space, options)
(RolloutWorker pid=45348) /simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/rllib/models/catalog.py:895: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
(RolloutWorker pid=45348)   prep = cls(observation_space, options)
(RolloutWorker pid=45348) /simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/rllib/connectors/agent/obs_preproc.py:37: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `pro

agent_timesteps_total: 4000
connector_metrics:
  ObsPreprocessorConnector_ms: 0.009772341738465012
  StateBufferConnector_ms: 0.008000994241365822
  ViewRequirementAgentConnector_ms: 0.2239401622485089
counters:
  num_agent_steps_sampled: 4000
  num_agent_steps_trained: 4000
  num_env_steps_sampled: 4000
  num_env_steps_trained: 4000
custom_metrics: {}
date: 2024-05-13_23-06-15
done: false
env_runner_results:
  connector_metrics:
    ObsPreprocessorConnector_ms: 0.009772341738465012
    StateBufferConnector_ms: 0.008000994241365822
    ViewRequirementAgentConnector_ms: 0.2239401622485089
  custom_metrics: {}
  episode_len_mean: 21.370967741935484
  episode_media: {}
  episode_return_max: 60.0
  episode_return_mean: 21.370967741935484
  episode_return_min: 8.0
  episode_reward_max: 60.0
  episode_reward_mean: 21.370967741935484
  episode_reward_min: 8.0
  episodes_this_iter: 186
  episodes_timesteps_total: 3975
  hist_stats:
    episode_lengths: [15, 43, 15, 11, 37, 45, 37, 34, 20, 58, 

In [12]:
import ray
from ray import train, tune
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.logger import pretty_print
ray.init(ignore_reinit_error=True)

config = PPOConfig().training(lr=tune.grid_search([0.01, 0.001, 0.0001]))

tuner = tune.Tuner(
    "PPO",
    run_config=train.RunConfig(
        stop={"episode_reward_mean": 150},
    ),
    param_space=config,
)

tuner.fit()

2024-05-13 23:35:04,488	ERROR tune_controller.py:1331 -- Trial task failed for trial PPO_None_563a1_00001
Traceback (most recent call last):
  File "/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/_private/worker.py", line 2623, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeou

ResultGrid<[
  Result(
    error='ActorDiedError',
    metrics={},
    path='/home/jianghaoning/ray_results/PPO_2024-05-13_23-34-50/PPO_None_563a1_00000_0_lr=0.0100_2024-05-13_23-34-50',
    filesystem='local',
    checkpoint=None
  ),
  Result(
    error='ActorDiedError',
    metrics={},
    path='/home/jianghaoning/ray_results/PPO_2024-05-13_23-34-50/PPO_None_563a1_00001_1_lr=0.0010_2024-05-13_23-34-50',
    filesystem='local',
    checkpoint=None
  ),
  Result(
    error='ActorDiedError',
    metrics={},
    path='/home/jianghaoning/ray_results/PPO_2024-05-13_23-34-50/PPO_None_563a1_00002_2_lr=0.0001_2024-05-13_23-34-50',
    filesystem='local',
    checkpoint=None
  )
]>

In [11]:
tuner = ray.tune.Tuner(
    "PPO",
    param_space=config,
    run_config=train.RunConfig(
        stop={"episode_reward_mean": 150},
        checkpoint_config=train.CheckpointConfig(checkpoint_at_end=True),
    ),
)

results = tuner.fit()

# Get the best result based on a particular metric.
best_result = results.get_best_result(metric="episode_reward_mean", mode="max")

# Get the best checkpoint corresponding to the best result.
best_checkpoint = best_result.checkpoint

2024-05-13 23:31:06,741	ERROR tune_controller.py:1331 -- Trial task failed for trial PPO_None_c95b8_00000
Traceback (most recent call last):
  File "/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/_private/worker.py", line 2623, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeou

RuntimeError: No best trial found for the given metric: episode_reward_mean. This means that no trial has reported this metric, or all values reported for this metric are NaN. To not ignore NaN values, you can set the `filter_nan_and_inf` arg to False.